In [121]:
import psycopg2
import pandas as pd
import datetime
import numpy as np
from unidecode import unidecode
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
import os
import sys
import pickle
import time

## FEATURE ENGINEERING EXP

In [122]:
# codificaciones = pd.read_excel(
#     r"cce/src/Data/Codificaciones.xlsx", sheet_name="codificaciones"
# )
codificaciones = pd.read_excel(
    r"/home/cnvdba/cce/src_dev/Data/Codificaciones.xlsx", sheet_name="codificaciones"
)
codificaciones.Code = codificaciones.Code.str.strip()
rename_columns = {
    "creation_date": "creationDate",
    "creation_time": "creationTime",
    "creditor_participant_code": "creditorParticipantCode",
    "debtor_participant_code": "debtorParticipantCode",
    "debtor_type_person": "debtorTypeOfPerson",
    "transaction_type": "transactionType",
    "debtor_id": "debtorId",
    "debtor_id_code": "debtorIdCode",
    "creditor_cci": "creditorCCI",
    "creditor_credit_card": "creditorCreditCard",
    "reason_code": "reasonCode",
    "response_code": "responseCode",
    "creditor_id": "creditorId",
    "creditor_id_code": "creditorIdCode",
}

dict_column_list = {
    "debtorParticipantCode": "participants",
    "creditorParticipantCode": "participants",
    "transactionType": "transaction_type",
    "currency": "currency",
    "channel": "channel",
    "responseCode": "response_code",
    "reasonCode": "reason_code",
}

In [123]:
_driver = "PostgreSQL ANSI(x64)"
_server = "localhost" #"10.201.4.25" 
_database = "coemcas"
_username = "coemcas"
_password = "C03$CMa5$2099"
_port = "5432"
PREDICTION_PIPELINE = False
NEW_VARIABLES_FLAG = True
def get_database_conection():
    connection = psycopg2.connect(
        host=_server,
        database=_database,
        user=_username,
        password=_password,
        port=_port
    )
    return connection


In [124]:
def data_ingestion_from_database():
    #TODO: REVERT SQL QUERY
    #Given the first unrun batch it finds, it retrieves all the transactions from the last 7 days (excluding the unrun batch) 
    if PREDICTION_PIPELINE:
        #TODO: temporarily remove duplicate rows (by pk)
        sql = """
            WITH run_id_to_process AS (
                SELECT r.run_id, r.run_date, r.run_end_datetime
                FROM fraud_model_run r
                WHERE r.run_start_datetime = (
                    SELECT MIN(r2.run_start_datetime)
                    FROM fraud_model_run r2
                    WHERE COALESCE(r2.run_process_status, 0) = 0
                )
            ),
            debtor_ids AS (
                SELECT DISTINCT m.debtor_id, r.run_id AS current_run_id, r.run_date AS current_run_date, r.run_end_datetime
                FROM run_id_to_process r
                INNER JOIN stage_ipf_message m ON r.run_id = m.run_id
            )
            SELECT DISTINCT ON (m.pk) m.*, d.current_run_id
            FROM stage_ipf_message m 
            INNER JOIN debtor_ids d ON m.debtor_id = d.debtor_id
            WHERE m.run_id = d.current_run_id
            OR (
                m.run_id != d.current_run_id
                AND m.creation_date BETWEEN d.current_run_date - interval '90 days' AND d.current_run_date
                AND (
                    m.creation_date < d.current_run_date 
                    OR (m.creation_date = d.current_run_date AND m.creation_time <= d.run_end_datetime::time)
                )
            );
        """
    else:
        #TODO: delete "limit 1000" and change "rn <= 20000" to "rn <= 200000"
        sql = """
            with debtor_ids as (
                select debtor_id, current_creation_date
                from (
                    select 
                        debtor_id, 
                        current_creation_date,
                        row_number() over (partition by current_creation_date order by random()) as rn
                    from (
                        select 
                            distinct m.debtor_id, 
                            m.creation_date as current_creation_date
                        from stage_ipf_message m
                        where m.creation_date between '2024-11-05' and '2024-11-11' limit 3000
                    ) distinct_pairs
                ) numbered_pairs
                where rn <= 3000
            ),
            filtered_rows as (
                select 
                    m.*, 
                    row_number() over (partition by m.pk) as row_num
                from stage_ipf_message m
                inner join debtor_ids d on m.debtor_id = d.debtor_id
                where m.creation_date between d.current_creation_date - interval '90 days' and d.current_creation_date
            )
            select *
            from filtered_rows
            where row_num = 1;
        """

    connection = None
    # Establish the database connection
    connection = get_database_conection()
    cursor = connection.cursor()
    
    print(f"EXEC - Started query to get AV_consolidado rows at {datetime.datetime.now()}")
    cursor.execute(sql)
    print(f"EXEC - Ended query (cursor.execute) to get AV_consolidado rows at {datetime.datetime.now()}")
    # Obtener los nombres de las columnas
    column_names = [desc[0] for desc in cursor.description]
    # Obtener los resultados como una lista de listas
    results = cursor.fetchall()
    print(f"EXEC - Ended query (cursor.fetchall) to get AV_consolidado rows at {datetime.datetime.now()}")
    cursor.close()
    if connection is not None:
        connection.close()
    # Convertir los resultados en un DataFrame
    AV_consolidado = pd.DataFrame(results, columns=column_names)

    return AV_consolidado

In [125]:
AV_consolidado = data_ingestion_from_database()

EXEC - Started query to get AV_consolidado rows at 2024-11-15 16:01:24.363784
EXEC - Ended query (cursor.execute) to get AV_consolidado rows at 2024-11-15 16:01:25.480758
EXEC - Ended query (cursor.fetchall) to get AV_consolidado rows at 2024-11-15 16:01:25.672762


In [126]:
if PREDICTION_PIPELINE:
    max_run_id = AV_consolidado.iloc[0]["current_run_id"]
else:
    max_run_id = "XXX"
drop_cols_av_cons = (
    ['log_timestamp_replica', 'last_modified', 'current_run_id'] 
    if PREDICTION_PIPELINE 
    else ['log_timestamp_replica', 'last_modified', 'row_num']
)
AV_consolidado = AV_consolidado.drop(columns=drop_cols_av_cons)
# AV_consolidado.shape

In [127]:
AV_consolidado.shape

(44897, 22)

In [128]:
import random

# Crear lista negra DUMMY

# Define a function to generate a list of unique strings
def generate_unique_number_strings(count, length):
    return ["".join(random.sample("0123456789", length)) for _ in range(count)]

# Initialize participant list and black_list dictionaries
participant_list = [
    "0002", "0003", "0007", "0009", "0011", "0018", "0023", "0035", 
    "0038", "0043", "0049", "0053", "0054", "0055", "0058", "0094", 
    "0096", "0801", "0802", "0803", "0805", "0806", "0808", "0809"
]
black_list = {"origen": {}, "destino": {}}

# Populate black_list with random unique number strings
for participant in participant_list:
    for bl in black_list:
        random_list = generate_unique_number_strings(10, 10)  # Generate 10 strings of 10 unique numbers
        black_list[bl][participant] = random_list

# Check results (printing the first few for brevity)
# for k, v in list(black_list['origen'].items())[:5]:
#     print(f"{k}: {v}")
# print("_____________________")
# for k, v in list(black_list['destino'].items())[:5]:
#     print(f"{k}: {v}")


In [129]:
# Crear personeria origen y destino DUMMY
unique_debtors = AV_consolidado['debtor_id'].unique()
unique_creditors = AV_consolidado['creditor_id'].unique()

personeria_options = ['Natural', 'Juridica']
new_columns = ["personeria_debtor_natural", "personeria_creditor_natural"]
debtor_personeria_mapping = {debtor: np.random.choice(personeria_options) for debtor in unique_debtors}
creditor_personeria_mapping = {creditor: np.random.choice(personeria_options) for creditor in unique_creditors}

AV_consolidado['personeria_debtor'] = AV_consolidado['debtor_id'].map(debtor_personeria_mapping)
AV_consolidado['personeria_creditor'] = AV_consolidado['creditor_id'].map(creditor_personeria_mapping)

# Crear monto de transaccion DUMMY

np.random.seed(0)  # Optional: for reproducibility
AV_consolidado['transaction_amount'] = np.random.uniform(100, 1000, size=len(AV_consolidado))
AV_consolidado.columns

Index(['pk', 'debtor_id', 'creditor_cci', 'creation_date', 'creation_time',
       'channel', 'currency', 'creditor_participant_code',
       'debtor_participant_code', 'debtor_type_person', 'transaction_type',
       'debtor_id_code', 'creditor_credit_card', 'reason_code',
       'response_code', 'creditor_id', 'creditor_id_code', 'message_id',
       'trace', 'instruction_id', 'run_id', 'same_customer_flag',
       'personeria_debtor', 'personeria_creditor', 'transaction_amount'],
      dtype='object')

In [130]:
AV_consolidado.personeria_debtor

0        Juridica
1         Natural
2         Natural
3         Natural
4         Natural
           ...   
44892     Natural
44893     Natural
44894     Natural
44895    Juridica
44896    Juridica
Name: personeria_debtor, Length: 44897, dtype: object

In [131]:
#NUEVA
# Renombrar columnas in-place
AV_consolidado.rename(columns=rename_columns, inplace=True)
# start = time.time()
for column_to_decode, value in dict_column_list.items():
    print(column_to_decode)
    # Filtrar codificaciones relevantes
    codificaciones_stage = codificaciones[codificaciones['List'] == value][["Code", "Value"]]
    codificaciones_dict = codificaciones_stage.set_index('Code')['Value'].to_dict()

    # Rellenar valores nulos en la columna a decodificar
    AV_consolidado[column_to_decode].fillna("", inplace=True)

    # Realizar el merge de forma más eficiente
    AV_consolidado['temp'] = AV_consolidado[column_to_decode].map(codificaciones_dict)

    if column_to_decode not in ["reasonCode"]:
        AV_consolidado['temp'] = AV_consolidado['temp'].fillna("invalid")
 
    # Insertar la columna mapeada en el lugar adecuado
    AV_consolidado[column_to_decode] = AV_consolidado['temp']
    AV_consolidado.drop('temp', axis=1, inplace=True)
# Renombrar columnas finales in-place
AV_consolidado.rename(
    columns={
        "debtorParticipantCode": "debtorParticipant",
        "creditorParticipantCode": "creditorParticipant",
    },
    inplace=True
)
# end = time.time()
# print("TOT TIME", end - start)

debtorParticipantCode
creditorParticipantCode
transactionType
currency


/tmp/ipykernel_3618309/4162895485.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AV_consolidado[column_to_decode].fillna("", inplace=True)
/tmp/ipykernel_3618309/4162895485.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

channel
responseCode
reasonCode


/tmp/ipykernel_3618309/4162895485.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AV_consolidado[column_to_decode].fillna("", inplace=True)
/tmp/ipykernel_3618309/4162895485.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [132]:
AV_consolidado.creationDate.drop_duplicates().sort_values().reset_index(drop=True)

0     2024-08-07
1     2024-08-08
2     2024-08-09
3     2024-08-10
4     2024-08-11
         ...    
92    2024-11-07
93    2024-11-08
94    2024-11-09
95    2024-11-10
96    2024-11-11
Name: creationDate, Length: 97, dtype: object

In [133]:
AV_consolidado["flag_invalid"] = (AV_consolidado.astype(str).apply(lambda x: x == "invalid").any(axis=1)).astype("uint8")
AV_consolidado_original_values_set_stage = AV_consolidado.copy()

In [134]:
AV_consolidado.reset_index(drop=True, inplace=True)
rows_dropped = (AV_consolidado_original_values_set_stage.shape[0] - AV_consolidado.shape[0])
if rows_dropped > 0:
    print(f"{rows_dropped} rows dropped because any value was invalid")
AV_consolidado.drop(["flag_invalid"], axis=1, inplace=True)

In [135]:
vars_to_discard = [
    "pk",
    "reasonCode",
    "run_id",
    "trace",
    "instruction_id",
    "message_id",
]  # 1

vars_to_feature_engineer = ["creationDate", "creationTime"]  # 2
vars_to_ohe = [
    "debtorTypeOfPerson",
    "debtorParticipant",
    "creditorParticipant",
    "transactionType",
    "currency",
    "channel",
    "responseCode",
    "personeria_debtor",
    "personeria_creditor",
]  # 3



#### FRECUENCIA

In [136]:
#FRECUENCIA
def create_frequency_features(df, new_cols, freq_days = [1, 7, 30, 90]):
    df = df.sort_values(by=["debtorId","creationDate","creationTime"]) #.dropna().copy()
    # df["creation_date_temp2"] = df["creation_date_temp"]
    df = df.reset_index(drop=True)
    df.set_index('creation_date_temp2', inplace=True)
    result_df = pd.DataFrame(index=df.index)
    day_intervals = df["day_interval"].unique()
    for di in day_intervals:
        df[f'di_is_{di}'] = (df['day_interval'] == di).astype(int)
    for days in freq_days:
        vals = [0] * len(df)
        for di in day_intervals:
            di_counts = (df.groupby('debtorId')[f'di_is_{di}']
                        .rolling(window=f'{days}d')
                        .sum()
                        .reset_index(level=0, drop=True))

            new_col = f"f{di}_{days}d"
            res = di_counts / days
            result_df[new_col] = res
            vals = [x + y for x, y in zip(vals, res)]
            new_cols.append(new_col)
        new_col = f"f{days}d"
        result_df[new_col] = vals
        new_cols.append(new_col)
        ################################### OG
        # result = (df.groupby('debtorId')
        #                 .rolling(window=f'{days}d', on='creation_date_temp2')
        #                 .creation_date_temp2
        #                 .count())  # Rolling count without resetting index
        # result = result.reset_index(level=0, drop=True)
        # new_col = f"f{days}d"
        # result_df[new_col] = result.values / days
        # new_cols.append(new_col)
        ##################################
    # print(result_df)
    result_df = result_df.reset_index(drop=True)
    for di in day_intervals:
        df.drop(columns=f'di_is_{di}', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, result_df], axis=1)
    return df, new_cols

#### CANTIDAD DE OPERACIONES DEL CLIENTE POR DIA Y POR CANAL

In [137]:
# CANTIDAD DE OPERACIONES DEL CLIENTE POR DIA Y POR CANAL
def rolling_total_count(group, days):
    return group.rolling(window=f'{days}D').count()

def create_frequency_per_channel(df, new_cols, freq_days=[1, 7, 30, 90]):
    df = df.sort_values(by=["debtorId", "creationDate", "creationTime"])
    df = df.reset_index(drop=True)
    df.set_index('creation_date_temp', inplace=True)
    df["clean_channel"] = df["channel"].astype(str).apply(unidecode).str.replace(" ", "_", regex="False").str.lower()
    channel_types = df["clean_channel"].unique()
    for channel in channel_types:
        df[f'channel_is_{channel}'] = (df['clean_channel'] == channel).astype(int)
    for days in freq_days:
        # Contar el total de eventos en la ventana de días
        total_counts = (
            df.groupby('debtorId')['clean_channel']
            .apply(lambda group: rolling_total_count(group, days))
            .reset_index(level=0, drop=True)
        )
        for channel in channel_types:
            # Contar eventos por canal en la ventana de días
            channel_counts = (
                df.groupby('debtorId')[f'channel_is_{channel}']
                .rolling(window=f'{days}D')
                .sum()
                .reset_index(level=0, drop=True)
            )

            # Calcular la proporción para ese canal y ventana de tiempo
            new_col = f'prop_{channel}_{days}d'
            df[new_col] = channel_counts / total_counts
            new_cols.append(new_col)
    for channel in channel_types:
        df.drop(columns=f'channel_is_{channel}', inplace=True)
    df.drop(columns=["clean_channel"], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df, new_cols

#### DNI DESTINO 

In [138]:
# DNI DESTINO 
def create_frequency_interaction_creditor_id(df, new_cols, freq_days = [1, 7, 30, 90]):
    df["creditorId"] = df["creditorId"].fillna("00000000")
    df = df.sort_values(by=["debtorId","creditorId", "creationDate","creationTime"]) #.dropna().copy()
    df = df.reset_index(drop=True)
    
    result_df = pd.DataFrame(index=df.index)
    for days in freq_days:
        # result = (df.groupby(['debtorId', 'creditorId'])
        #                     .rolling(window=f'90d', on='creation_date_temp')
        #                     .creation_date_temp
        #                     .count().reset_index(drop=True))
        result = (df.groupby(['debtorId', 'creditorId'])
                .rolling(window=f'{days}d', on='creation_date_temp')
                .creation_date_temp
                .count())
        result = result.reset_index(level=[0], drop=True)
        new_col = f"f{days}d_to_creditor"
        result_df[new_col] = result.values / days
        new_cols.append(new_col)
    # print(result_df)
    result_df = result_df.reset_index(drop=True)
    df = pd.concat([df, result_df], axis=1)
    return df, new_cols   

#### CUENTAS CON ABONOS DE DIFERENTES ORIGENES

In [139]:
#CUENTAS CON ABONOS DE DIFERENTES ORIGENES
def create_unique_debtors_per_creditor(df, new_cols, freq_days = [1, 7, 30, 90]):    
    # Sort by CreditorCCI, debtorId, and transaction date to ensure chronological order
    df["creditorId"] = df["creditorId"].fillna("00000000")
    df = df.sort_values(by=["creditorId", "creationDate","creationTime"])
    df = df.reset_index(drop=True)
    
    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame(index=df.index)

    # Use a rolling window per creditor to get unique debtorIds
    # result = (df.groupby('creditorId')
    #             .rolling(window=f'{window_days}d', on='creation_date_temp')
    #             .apply(lambda x: x['debtorId'].unique(), raw=False)
    #             )
    df["hash_id"] = pd.factorize(df["debtorId"])[0]
    # Crear ambos diccionarios en un solo bucle
    # hash_in, hash_out = {}, {}
    # for _, row in df.iterrows():
    #     debtor_id = row["debtorId"]
    #     hash_id = row["hash_id"]
    #     hash_in[debtor_id] = hash_id
    #     hash_out[hash_id] = debtor_id
    for days in freq_days:
        print(days)
        # result = df.groupby('creditorId').apply(lambda x: x.rolling(window=f'{days}d', on='creation_date_temp').hash_id.apply(lambda y: y.nunique() ))
        result = (df.groupby('creditorId').rolling(window=f'{days}d', on='creation_date_temp')['hash_id'].apply(lambda y: y.nunique()))
        # Reset the index to avoid issues with multi-indexing
        # result = result.reset_index(level=[0], drop=True)
        result = result.reset_index(level=[0], drop=True).values

        # Add the result to the original DataFrame
        new_col = f"unique_debtors_past_{days}d"
        result_df[new_col] = result
        new_cols.append(new_col)
        # Concatenate with the original DataFrame
    df = pd.concat([df, result_df], axis=1)
    df.drop(columns=["hash_id"], inplace=True)
    
    return df, new_cols

#### PROPORCION DE MONTOS

In [140]:
def amount_proportion(df, new_cols, freq_days = [1, 7, 30, 90]):
    df = df.sort_values(by=["debtorId", "creationDate","creationTime"]) #.dropna().copy()
    df = df.reset_index(drop=True)
    result_df = pd.DataFrame(index=df.index)
    for days in freq_days:
        result = (df.groupby(['debtorId'])
                .rolling(window=f'{days}d', on='creation_date_temp')
                .transaction_amount
                .sum())
        result = result.reset_index(level=[0], drop=True)
        new_col = f"prop{days}d_amount"
        result_df[new_col] = df.transaction_amount / result.values
        new_cols.append(new_col)
    # print(result_df)
    result_df = result_df.reset_index(drop=True)
    df = pd.concat([df, result_df], axis=1)
    return df, new_cols   

In [141]:
def categorize_day_interval(hour):
    """
    Categorize the given hour into specific day intervals.

    Args:
        hour (int or str): The hour to be categorized. It can be an integer or a string representation of an integer.

    Returns:
        str: The category corresponding to the input hour.

    """
    # Zero-padding if needed
    hour = str(hour).zfill(6)  
    # Extract the hour part
    hour = int(hour[:2])  
    # Define day interval categories
    if hour >= 0 and hour < 6:
        return 'early morning'
    elif hour >= 6 and hour < 12:
        return 'morning'
    elif hour >= 12 and hour < 18:
        return 'afternoon'
    else:  # hour >= 18 or hour < 24
        return 'evening'

In [142]:
AV_consolidado["day_interval"] = AV_consolidado["creationTime"].apply(categorize_day_interval)

In [143]:
AV_consolidado["creation_date_temp"] = pd.to_datetime(AV_consolidado["creationDate"])
temp = AV_consolidado.copy(deep=True)

In [144]:
#NEW VARIABLES
if NEW_VARIABLES_FLAG:
    AV_consolidado["creation_date_temp2"] = pd.to_datetime(AV_consolidado["creationDate"])
    AV_consolidado["creation_date_temp"] = pd.to_datetime(AV_consolidado["creationDate"])
    #in this exact order, to only create "creation_date_temp" once
    AV_consolidado, new_columns = create_frequency_features(AV_consolidado, new_columns)
    print("first")
    AV_consolidado, new_columns = create_frequency_interaction_creditor_id(AV_consolidado, new_columns)
    print("second")
    AV_consolidado, new_columns = create_unique_debtors_per_creditor(AV_consolidado, new_columns)
    print("third")
    AV_consolidado, new_columns = amount_proportion(AV_consolidado, new_columns)
    print("fourth")
    AV_consolidado, new_columns = create_frequency_per_channel(AV_consolidado, new_columns)
    print("fifth")
    AV_consolidado_original_values_set_stage = AV_consolidado.copy()

first
second
1
7
30
90
third
fourth
fifth


In [145]:
# NEW
# 1 vars_to_discard
AV_consolidado = AV_consolidado.drop(columns=vars_to_discard)

# 2 vars_to_feature_engineer
AV_consolidado = AV_consolidado.rename(
    columns={
        "creationDate": "creationDate_stage",
        "creationTime": "creationTime_stage",
    }
)

# Conversión de fechas y horas
AV_consolidado["creationDate_stage"] = pd.to_datetime(AV_consolidado["creationDate_stage"])
AV_consolidado["creationTime_stage"] = AV_consolidado["creationTime_stage"].astype(str).str.replace(":", "", regex=False)
AV_consolidado["creationTime"] = pd.to_datetime(AV_consolidado["creationTime_stage"], format="%H%M%S").dt.time

# Creación de características cíclicas
def create_cyclic_features(df):
    df["hourSin"] = np.sin(2 * np.pi * df["creationTime"].apply(lambda x: x.hour) / 24.0)
    df["hourCos"] = np.cos(2 * np.pi * df["creationTime"].apply(lambda x: x.hour) / 24.0)
    df["dayOfYearSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.dayofyear / 365.0)
    df["dayOfYearCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.dayofyear / 365.0)
    df["dayOfMonthSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.day / 31.0)
    df["dayOfMonthCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.day / 31.0)
    df["dayOfWeekSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.weekday / 7.0)
    df["dayOfWeekCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.weekday / 7.0)
    df["monthSin"] = np.sin(2 * np.pi * df["creationDate_stage"].dt.month / 12.0)
    df["monthCos"] = np.cos(2 * np.pi * df["creationDate_stage"].dt.month / 12.0)
    return df


vars_to_feature_engineer = ["creationTime"]  # 2

AV_consolidado = create_cyclic_features(AV_consolidado)

# Eliminación de columnas intermedias
AV_consolidado = AV_consolidado.drop(columns=["creationDate_stage", "creationTime_stage", *vars_to_feature_engineer])

# 3 vars_to_ohe
AV_consolidado_3_stage = AV_consolidado.copy()


In [146]:
# Cambiar formato a "snake_case" sin tildes
for column in vars_to_ohe:
    AV_consolidado_3_stage[column] = (
        AV_consolidado_3_stage[column]
        .astype(str)
        .apply(unidecode)
        .str.replace(" ", "_", regex=False)
        .str.lower()
    )

In [147]:
vars_to_ohe

['debtorTypeOfPerson',
 'debtorParticipant',
 'creditorParticipant',
 'transactionType',
 'currency',
 'channel',
 'responseCode',
 'personeria_debtor',
 'personeria_creditor']

In [148]:
if PREDICTION_PIPELINE:
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
else:
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore", drop="first")
ohe.fit(AV_consolidado_3_stage[vars_to_ohe])

/home/cnvdba/miniconda3/envs/mvp1_shap/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
              sparse_output=False)

In [149]:
ohe.categories_

[array(['none'], dtype=object),
 array(['alfin_banco_s.a.', 'banbif', 'banco_de_la_nacion',
        'banco_falabella', 'banco_pichincha', 'banco_ripley', 'bbva',
        'bcp', 'caja_arequipa', 'caja_cusco', 'caja_huancayo', 'caja_ica',
        'caja_piura', 'caja_trujillo', 'crediscotia_financiera',
        'financiera_oh', 'gnb', 'interbank', 'invalid', 'mi_banco',
        'scotiabank'], dtype=object),
 array(['alfin_banco_s.a.', 'banbif', 'banco_de_la_nacion',
        'banco_falabella', 'banco_pichincha', 'banco_ripley', 'bbva',
        'bcp', 'caja_arequipa', 'caja_cusco', 'caja_huancayo', 'caja_ica',
        'caja_piura', 'caja_trujillo', 'citibank', 'comercio',
        'crediscotia_financiera', 'financiera_efectiva', 'financiera_oh',
        'gnb', 'interbank', 'invalid', 'mi_banco', 'scotiabank'],
       dtype=object),
 array(['ordinary_transfer', 'payments_to_account_card'], dtype=object),
 array(['dollars', 'soles'], dtype=object),
 array(['atm', 'banca_movil', 'invalid', 'web

In [150]:
AV_consolidado_3_ohe_stage = pd.DataFrame(ohe.transform(AV_consolidado_3_stage[vars_to_ohe]),columns=ohe.get_feature_names_out(vars_to_ohe))

In [151]:
AV_consolidado = pd.concat([AV_consolidado_3_stage.drop(vars_to_ohe, axis=1), AV_consolidado_3_ohe_stage],axis=1)

In [152]:
AV_consolidado_3_stage.head()

,debtorId,creditorCCI,channel,currency,creditorParticipant,debtorParticipant,debtorTypeOfPerson,transactionType,debtorIdCode,creditorCreditCard,...,hourSin,hourCos,dayOfYearSin,dayOfYearCos,dayOfMonthSin,dayOfMonthCos,dayOfWeekSin,dayOfWeekCos,monthSin,monthCos
0,000000018298,00253519440189506733,invalid,soles,bcp,banco_de_la_nacion,none,ordinary_transfer,4,None,...,-1.000000e+00,-1.836970e-16,-0.668064,-0.744104,0.651372,-0.758758,0.000000,1.000000,-0.866025,-0.5
1,000000018298,00253519137531001137,invalid,soles,bcp,banco_de_la_nacion,none,ordinary_transfer,4,None,...,-7.071068e-01,-7.071068e-01,-0.785650,-0.618671,-0.968077,-0.250653,0.433884,-0.900969,-0.866025,-0.5
2,000000018298,00253519410805205636,invalid,soles,bcp,banco_de_la_nacion,none,ordinary_transfer,4,None,...,-2.588190e-01,-9.659258e-01,-0.796183,-0.605056,-0.998717,-0.050649,-0.433884,-0.900969,-0.866025,-0.5
3,000000018298,00253519218341307731,invalid,soles,bcp,banco_de_la_nacion,none,ordinary_transfer,4,None,...,1.224647e-16,-1.000000e+00,-0.816538,-0.577292,-0.937752,0.347305,-0.781831,0.623490,-0.866025,-0.5
4,000000018298,00253519699567404535,invalid,soles,bcp,banco_de_la_nacion,none,ordinary_transfer,4,None,...,9.659258e-01,-2.588190e-01,-0.826354,-0.563151,-0.848644,0.528964,0.000000,1.000000,-0.866025,-0.5


In [153]:
AV_consolidado[AV_consolidado_3_ohe_stage.columns] = AV_consolidado[AV_consolidado_3_ohe_stage.columns].astype("uint8")
AV_consolidado_completed = pd.DataFrame()
AV_consolidado_completed = pd.concat([AV_consolidado_completed, AV_consolidado], axis=0, ignore_index=True)

In [154]:
AV_consolidado_original = pd.DataFrame()
AV_consolidado_original_stage = AV_consolidado.copy()
AV_consolidado_original = pd.concat([AV_consolidado_original, AV_consolidado_original_stage],axis=0,ignore_index=True,)
AV_consolidado_original_values_set = pd.DataFrame()
AV_consolidado_original_values_set = pd.concat([AV_consolidado_original_values_set, AV_consolidado_original_values_set_stage],axis=0,ignore_index=True,)


In [155]:
# Fill nulls
AV_consolidado_completed.fillna(value=0, inplace=True)

# Set all binary to uint8 (except the ones that are uint8 already)
id_columns = [
    "debtorId",
    "debtorIdCode",
    "creditorCCI",
    "creditorCreditCard",
    "creditorId",
    "creditorIdCode",
]

excluded_columns = (
    [
        "hourSin",
        "hourCos",
        "dayOfYearSin",
        "dayOfYearCos",
        "dayOfMonthSin",
        "dayOfMonthCos",
        "dayOfWeekSin",
        "dayOfWeekCos",
        "monthSin",
        "monthCos",
        "same_customer_flag",
        "transaction_amount",
        "day_interval"
    ]
    + AV_consolidado_completed.select_dtypes(include="uint8").columns.to_list()
    + id_columns 
)
if NEW_VARIABLES_FLAG:
    excluded_columns +=  new_columns


In [156]:
new_columns

['personeria_debtor_natural',
 'personeria_creditor_natural',
 'fevening_1d',
 'fafternoon_1d',
 'fmorning_1d',
 'fearly morning_1d',
 'f1d',
 'fevening_7d',
 'fafternoon_7d',
 'fmorning_7d',
 'fearly morning_7d',
 'f7d',
 'fevening_30d',
 'fafternoon_30d',
 'fmorning_30d',
 'fearly morning_30d',
 'f30d',
 'fevening_90d',
 'fafternoon_90d',
 'fmorning_90d',
 'fearly morning_90d',
 'f90d',
 'f1d_to_creditor',
 'f7d_to_creditor',
 'f30d_to_creditor',
 'f90d_to_creditor',
 'unique_debtors_past_1d',
 'unique_debtors_past_7d',
 'unique_debtors_past_30d',
 'unique_debtors_past_90d',
 'prop1d_amount',
 'prop7d_amount',
 'prop30d_amount',
 'prop90d_amount',
 'prop_invalid_1d',
 'prop_banca_movil_1d',
 'prop_web_1d',
 'prop_atm_1d',
 'prop_invalid_7d',
 'prop_banca_movil_7d',
 'prop_web_7d',
 'prop_atm_7d',
 'prop_invalid_30d',
 'prop_banca_movil_30d',
 'prop_web_30d',
 'prop_atm_30d',
 'prop_invalid_90d',
 'prop_banca_movil_90d',
 'prop_web_90d',
 'prop_atm_90d']

In [157]:
# Get a list of column names excluding 'excluded_columns' and id columns
binary_columns = list(set(AV_consolidado_completed.columns).difference(excluded_columns))
print("BINARY", binary_columns)

# Convert the binary columns to uint8
AV_consolidado_completed[binary_columns] = AV_consolidado_completed[binary_columns].astype("uint8")


BINARY []


In [158]:
# # Export AV_consolidado_completed
# output_file_path = "cce_ipf_message_feature_engineering.pickle"
# AV_consolidado_completed.to_pickle(output_file_path)

# # Export AV_consolidado_original
# output_file_path = "cce_ipf_message_original.pickle"
# AV_consolidado_original.to_pickle(output_file_path)

# # Export AV_consolidado_original_values_set
# output_file_path = "cce_ipf_message_original_values_set.pickle"
# AV_consolidado_original_values_set.to_pickle(output_file_path)
# print("end of execution")


In [159]:
AV_consolidado_original_values_set.shape

(44897, 74)

In [160]:
AV_consolidado_completed.shape

(44897, 118)

## FEATURE ENGINEERING RATIOS

In [161]:
def generate_combinations(input_list):
    """
    Generate all possible combinations of elements from the given input list.

    The function generates combinations by first including each element in a separate list,
    and then combining two elements at a time to form new combinations.

    Args:
        input_list (list): The list of elements from which to generate combinations.

    Returns:
        list: A list containing all possible combinations of elements.
    """
    
    output_list = []

    for i in range(len(input_list)):
        # Add each element in a list of 1 item
        output_list.append([input_list[i]])

        for j in range(i + 1, len(input_list)):
            # Add combinations of two elements
            output_list.append([input_list[i], input_list[j]])

    return output_list

def categorize_hour(hour):
    """
    Categorize the given hour into specific time intervals.

    Args:
        hour (int or str): The hour to be categorized. It can be an integer or a string representation of an integer.

    Returns:
        str: The category corresponding to the input hour.

    """
    # Zero-padding if needed
    hour = str(hour).zfill(6)  
    # Extract the hour part
    hour = int(hour[:2])  
    # Define time interval categories
    if hour >= 0 and hour < 3:
        return '00 to 03'
    elif hour >= 3 and hour < 6:
        return '03 to 06'
    elif hour >= 6 and hour < 9:
        return '06 to 09'
    elif hour >= 9 and hour < 12:
        return '09 to 12'
    elif hour >= 12 and hour < 15:
        return '12 to 15'
    elif hour >= 15 and hour < 18:
        return '15 to 18'
    elif hour >= 18 and hour < 21:
        return '18 to 21'
    else:  # hour >= 21 or hour < 24
        return '21 to 00'



In [162]:
# Define a list of columns to keep in the resulting DataFrame
lista = [
    "debtorId",
    "creditorCCI",
    "run_id",
    "creditorParticipant",
    "currency",
    "channel",
    "responseCode",
    "debtorParticipant",
    "creationDate",
    "creationTime",
    "time_interval",
    "Weekday",
]

# Generate all possible combinations of columns to be used for ratio calculations

input_list = [
    "creditorParticipant",
    "currency",
    "channel",
    "responseCode",
    "debtorParticipant",
    "Weekday",
    "time_interval",
    "creditorCCI",
]

if NEW_VARIABLES_FLAG:
    lista += ["creditorId","personeria_creditor"]
    input_list += ["creditorId", "personeria_creditor"]

output_list = generate_combinations(input_list)

In [163]:
#antes = 45 
# dsps
len(output_list)

55

In [164]:
directory_path = ""
# Define the filenames for feature engineering and original values set DataFrames
file_name_feature_engineering = "cce_ipf_message_feature_engineering.pickle"
file_name_original_values_set = "cce_ipf_message_original_values_set.pickle"

# Build the file paths for the feature engineering and original values set DataFrames
file_path_fe = os.path.join(directory_path, file_name_feature_engineering)
file_path_o_vs = os.path.join(directory_path, file_name_original_values_set)

In [165]:
# # Export AV_consolidado_completed
# output_file_path = "cce_ipf_message_feature_engineering.pickle"
# AV_consolidado_completed.to_pickle(output_file_path)

# # Export AV_consolidado_original
# output_file_path = "cce_ipf_message_original.pickle"
# AV_consolidado_original.to_pickle(output_file_path)

# # Export AV_consolidado_original_values_set
# output_file_path = "cce_ipf_message_original_values_set.pickle"
# AV_consolidado_original_values_set.to_pickle(output_file_path)
# print("end of execution")

In [166]:
# Read the original values set DataFrame
# AV_o_vs = pd.read_pickle(file_path_o_vs)
AV_o_vs = AV_consolidado_original_values_set
# Filter and preprocess the original values set DataFrame
AV_o_vs = AV_o_vs[AV_o_vs["transactionType"] == "Ordinary Transfer"]
# AV_o_vs = AV_o_vs[AV_o_vs['debtorParticipant'].isin(code) | AV_o_vs['creditorParticipant'].isin(code)]
AV_o_vs["creationDate"] = pd.to_datetime(AV_o_vs["creationDate"])
df = AV_o_vs.copy()
df["Weekday"] = df["creationDate"].apply(lambda x: x.weekday()).astype(object)
df["time_interval"] = df["creationTime"].apply(categorize_hour)



/tmp/ipykernel_3618309/1182050474.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AV_o_vs["creationDate"] = pd.to_datetime(AV_o_vs["creationDate"])


In [167]:
AV_consolidado_original_values_set.columns

Index(['pk', 'debtorId', 'creditorCCI', 'creationDate', 'creationTime',
       'channel', 'currency', 'creditorParticipant', 'debtorParticipant',
       'debtorTypeOfPerson', 'transactionType', 'debtorIdCode',
       'creditorCreditCard', 'reasonCode', 'responseCode', 'creditorId',
       'creditorIdCode', 'message_id', 'trace', 'instruction_id', 'run_id',
       'same_customer_flag', 'personeria_debtor', 'personeria_creditor',
       'transaction_amount', 'day_interval', 'fevening_1d', 'fafternoon_1d',
       'fmorning_1d', 'fearly morning_1d', 'f1d', 'fevening_7d',
       'fafternoon_7d', 'fmorning_7d', 'fearly morning_7d', 'f7d',
       'fevening_30d', 'fafternoon_30d', 'fmorning_30d', 'fearly morning_30d',
       'f30d', 'fevening_90d', 'fafternoon_90d', 'fmorning_90d',
       'fearly morning_90d', 'f90d', 'f1d_to_creditor', 'f7d_to_creditor',
       'f30d_to_creditor', 'f90d_to_creditor', 'unique_debtors_past_1d',
       'unique_debtors_past_7d', 'unique_debtors_past_30d',
       

In [168]:
len(AV_consolidado_original_values_set)

44897

In [169]:
len(AV_consolidado_original_values_set[AV_consolidado_original_values_set["transactionType"] != "Ordinary Transfer"])

1002

In [170]:
def Ratio(dd1, output_list, lista, days_back): #revisar la fecha
    """
    Calculate ratios based on cumulative counts for specified columns in the input DataFrame.
 
    The function takes a DataFrame dd1 and calculates ratios for combinations of columns specified in output_list.
    The calculation is performed based on cumulative counts of each combination relative to a total count.
 
    Args:
        dd1 (pandas.DataFrame): The input DataFrame containing the data.
        output_list (list): A list of lists, where each inner list represents a combination of columns for which ratios
                            are to be calculated based on cumulative counts. The columns in each combination should be
                            present in the DataFrame dd1.
        lista (list): A list of column names from the DataFrame dd1 that will be used to extract data.
 
    Returns:
        pandas.DataFrame: A DataFrame containing the calculated ratios for each combination specified in output_list.
    """
    creditor =["debtorId"]
    dd2_completed = pd.DataFrame()
 
    total_days = dd1.creationDate.drop_duplicates().sort_values().reset_index(drop=True)
    # print(total_days)
    if PREDICTION_PIPELINE:
        training_days = [total_days.max()]
    else:
        training_days = total_days[total_days>= '2024-11-05'].reset_index(drop=True)
        
    for i, _day in enumerate(training_days):
        print(f"processing day {i+1} of {len(training_days)}: {_day}")
        dd2=dd1[(dd1.creationDate <= _day) & (dd1.creationDate >= _day - pd.Timedelta(days=90))][lista]\
            .sort_values(by=["debtorId","creationDate","creationTime"]).dropna().copy()
        dd2["count_cci"]=dd2.groupby(["debtorId"]).cumcount() + 1
        for i in output_list:
            dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount() + 1 #more precise
            dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
        #TODO: EL FILTRO DE RUN_ID YA NO SERIA, SERIA EL MAX CREATION DATE
        print("SIZE DD2 (before)", days_back, dd2.shape)
        if PREDICTION_PIPELINE:
            dd2 = dd2[dd2.run_id == max(dd2.run_id)]
        else:
            dd2 = dd2[dd2.creationDate == _day]
        print("SIZE DD2 (after)", days_back, dd2.shape)
        dd2_completed = pd.concat([dd2_completed, dd2], axis=0, ignore_index=False)
        dd2_completed = pd.concat([dd2_completed, dd2], axis=0, ignore_index=False)
    return(dd2_completed.filter(like="ratio"))

In [171]:
# Calculate ratios based on cumulative counts for specified columns
df["creditorId"] = df["creditorId"].fillna("00000000")
ratios_df = Ratio(df, output_list, lista, 90)
print(ratios_df.shape)
ratios_df.dropna(inplace=True)
print(ratios_df.shape)



processing day 1 of 7: 2024-11-05 00:00:00


/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
/tmp/ipykernel_3618309/2758591593.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount() + 1 #more precise
/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

SIZE DD2 (before) 90 (39531, 125)
SIZE DD2 (after) 90 (953, 125)
processing day 2 of 7: 2024-11-06 00:00:00


/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
/tmp/ipykernel_3618309/2758591593.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount() + 1 #more precise
/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

SIZE DD2 (before) 90 (40182, 125)
SIZE DD2 (after) 90 (815, 125)
processing day 3 of 7: 2024-11-07 00:00:00


...

/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
/tmp/ipykernel_3618309/2758591593.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount() + 1 #more precise
/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

SIZE DD2 (before) 90 (42066, 125)
SIZE DD2 (after) 90 (503, 125)
processing day 7 of 7: 2024-11-11 00:00:00


/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
/tmp/ipykernel_3618309/2758591593.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount() + 1 #more precise
/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

SIZE DD2 (before) 90 (42497, 125)
SIZE DD2 (after) 90 (743, 125)
(10634, 55)
(10634, 55)


/tmp/ipykernel_3618309/2758591593.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_cumcount"]=dd2.groupby(creditor+i).cumcount() + 1 #more precise
/tmp/ipykernel_3618309/2758591593.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dd2['_'.join(map(str, i))+"_ratio"]=dd2['_'.join(map(str, i))+"_cumcount"]/dd2["count_cci"]
/tmp/ipykernel_3618309/2758591593.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [172]:
# Read the feature engineering DataFrame
# AV_fe = pd.read_pickle(file_path_fe)
AV_fe = AV_consolidado_completed
AV_fe['in_black_debtor'] = AV_fe['debtorId'].apply(lambda x: any(x in bl for bl in black_list['origen'].values()))
AV_fe['in_black_creditor'] = AV_fe['debtorId'].apply(lambda x: any(x in bl for bl in black_list['destino'].values()))
AV_fe = AV_fe.drop(["debtorIdCode", "creditorCreditCard", "creditorId", "creditorIdCode"], axis=1)

In [173]:
#TODO: later after training/predicting, like with same_customer_flag, add the rows back. but now, with score = 99.
# if some rows have same_customer_flag = 'M' but either debtor or creditor are in any black list, that row has score = 99.

black_df = AV_fe[AV_fe['in_black_debtor'] | AV_fe['in_black_creditor']]  # Rows where either in_origen or in_destino is True
good_df = AV_fe[~(AV_fe['in_black_debtor'] | AV_fe['in_black_creditor'])]  # Rows where both in_origen and in_destino are False
AV_fe = good_df

In [174]:
# Merge the calculated ratios DataFrame and the feature engineering DataFrame
final = pd.merge(ratios_df, AV_fe, left_index=True, right_index=True)

final_same = final[final["same_customer_flag"] == 'M']
final_diff = final[final["same_customer_flag"] != 'M']
print(final.shape)
print(final_same.shape)
print(final_diff.shape)
final = final_diff
print(final.shape)
# Save the final DataFrame to a pickle file
directory_path = ''
# final.to_pickle(os.path.join(directory_path, "df_output.pickle"))

(10634, 171)
(2326, 171)
(8308, 171)
(8308, 171)


## TRAIN MODEL

In [175]:
RS = 12494328
train_data_file_name = "df_output.pickle"
#AV_train = pd.read_pickle(train_data_file_name)
AV_train = final.copy(deep=True)
# print("nasss", AV_train.isna().sum() / AV_train.shape[0])
# print(AV_train)
# Explore train data
print(f"AV_train shape: {AV_train.shape}")
# print(f"AV_train columns: {AV_train.columns}")
# print(f"creditorCCI únicos: {AV_train.creditorCCI.nunique()}")
# print("Top 10 creditor CCI con más operaciones AV:")
# print(AV_train.creditorCCI.value_counts().sort_values(ascending=False).head(10))
AV_train_nunique = AV_train.nunique()
cols_to_drop_unique_value = AV_train_nunique[AV_train_nunique == 1].index.to_list()
# drop columns with unique value
if len(cols_to_drop_unique_value) > 0:
    print("dropped", cols_to_drop_unique_value)
    # print(cols_to_drop_unique_value.index)
    AV_train.drop(cols_to_drop_unique_value, axis=1, inplace=True)
print(AV_train.shape)
AV_train.dropna(inplace=True)
print(AV_train.shape)

args_5 = {"random_state": RS, "contamination": 0.0196}
id_cols = ["debtorId", "creditorCCI", "in_black_debtor", "in_black_creditor"]

AV_train shape: (8308, 171)
dropped ['monthSin', 'monthCos', 'debtorParticipant_banco_ripley', 'debtorParticipant_caja_huancayo', 'debtorParticipant_caja_ica', 'debtorParticipant_caja_trujillo', 'creditorParticipant_banco_ripley', 'creditorParticipant_financiera_efectiva', 'transactionType_payments_to_account_card', 'in_black_debtor', 'in_black_creditor']
(8308, 160)
(8308, 160)


In [176]:
variables = [
    "creditorParticipant_ratio",
    "creditorParticipant_currency_ratio",
    "creditorParticipant_channel_ratio",
    "creditorParticipant_responseCode_ratio",
    "creditorParticipant_debtorParticipant_ratio",
    "creditorParticipant_Weekday_ratio",
    "creditorParticipant_time_interval_ratio",
    "creditorParticipant_creditorCCI_ratio",
    "currency_ratio",
    "currency_channel_ratio",
    "currency_responseCode_ratio",
    "currency_debtorParticipant_ratio",
    "currency_Weekday_ratio",
    "currency_time_interval_ratio",
    "currency_creditorCCI_ratio",
    "channel_ratio",
    "channel_responseCode_ratio",
    "channel_debtorParticipant_ratio",
    "channel_Weekday_ratio",
    "channel_time_interval_ratio",
    "channel_creditorCCI_ratio",
    "responseCode_ratio",
    "responseCode_debtorParticipant_ratio",
    "responseCode_Weekday_ratio",
    "responseCode_time_interval_ratio",
    "responseCode_creditorCCI_ratio",
    "debtorParticipant_ratio",
    "debtorParticipant_Weekday_ratio",
    "debtorParticipant_time_interval_ratio",
    "debtorParticipant_creditorCCI_ratio",
    "Weekday_ratio",
    "Weekday_time_interval_ratio",
    "Weekday_creditorCCI_ratio",
    "time_interval_ratio",
    "time_interval_creditorCCI_ratio",
    "creditorCCI_ratio",
    "debtorId",
    "creditorCCI",
    "hourSin",
    "hourCos",
    "dayOfYearSin",
    "dayOfYearCos",
    "dayOfMonthSin",
    "dayOfMonthCos",
    "dayOfWeekSin",
    "dayOfWeekCos",
    "debtorParticipant_bcp",
    "debtorParticipant_interbank",
    "debtorParticipant_citibank",
    "debtorParticipant_scotiabank",
    "debtorParticipant_bbva",
    "debtorParticipant_banco_de_la_nacion",
    "debtorParticipant_comercio",
    "debtorParticipant_banco_pichincha",
    "debtorParticipant_banbif",
    "debtorParticipant_crediscotia_financiera",
    "debtorParticipant_mi_banco",
    "debtorParticipant_gnb",
    "debtorParticipant_banco_falabella",
    "debtorParticipant_banco_ripley",
    "debtorParticipant_alfin_banco_s.a.",
    "debtorParticipant_financiera_oh",
    "debtorParticipant_financiera_efectiva",
    "debtorParticipant_caja_piura",
    "debtorParticipant_caja_trujillo",
    "debtorParticipant_caja_arequipa",
    "debtorParticipant_caja_sullana",
    "debtorParticipant_caja_cusco",
    "debtorParticipant_caja_huancayo",
    "debtorParticipant_caja_ica",
    "debtorParticipant_invalid",
    "creditorParticipant_bcp",
    "creditorParticipant_interbank",
    "creditorParticipant_citibank",
    "creditorParticipant_scotiabank",
    "creditorParticipant_bbva",
    "creditorParticipant_banco_de_la_nacion",
    "creditorParticipant_comercio",
    "creditorParticipant_banco_pichincha",
    "creditorParticipant_banbif",
    "creditorParticipant_crediscotia_financiera",
    "creditorParticipant_mi_banco",
    "creditorParticipant_gnb",
    "creditorParticipant_banco_falabella",
    "creditorParticipant_banco_ripley",
    "creditorParticipant_alfin_banco_s.a.",
    "creditorParticipant_financiera_oh",
    "creditorParticipant_financiera_efectiva",
    "creditorParticipant_caja_piura",
    "creditorParticipant_caja_trujillo",
    "creditorParticipant_caja_arequipa",
    "creditorParticipant_caja_sullana",
    "creditorParticipant_caja_cusco",
    "creditorParticipant_caja_huancayo",
    "creditorParticipant_caja_ica",
    "creditorParticipant_invalid",
    "currency_soles",
    "channel_banca_movil",
    "channel_invalid",
    "channel_web",
    "responseCode_rejected",
]
if NEW_VARIABLES_FLAG:
    new_ratios = [
        "creditorParticipant_creditorId_ratio",
        "currency_creditorId_ratio",
        "channel_creditorId_ratio",
        "responseCode_creditorId_ratio",
        "debtorParticipant_creditorId_ratio",
        "Weekday_creditorId_ratio",
        "time_interval_creditorId_ratio",
        "creditorCCI_creditorId_ratio",
        "creditorId_ratio",
        #### personeria ratios
        "creditorParticipant_personeria_creditor_ratio",
        "currency_personeria_creditor_ratio",
        "channel_personeria_creditor_ratio",
        "responseCode_personeria_creditor_ratio",
        "debtorParticipant_personeria_creditor_ratio",
        "Weekday_personeria_creditor_ratio",
        "time_interval_personeria_creditor_ratio",
        "creditorCCI_personeria_creditor_ratio",
        "creditorId_personeria_creditor_ratio",
        "personeria_creditor_ratio",
    ]
    new_columns += ["transaction_amount", "in_black_debtor", "in_black_creditor"]
    variables += new_columns + new_ratios

for col in variables:
    if col not in AV_train.columns:
        print("added as 0", col)
        AV_train[col] = 0

for x in AV_train.columns:
    if x not in variables:
        print("dropping...", x)
AV_train = AV_train[variables]
# print("variables train", AV_train.columns.tolist())

added as 0 debtorParticipant_citibank
added as 0 debtorParticipant_comercio
added as 0 debtorParticipant_banco_ripley
added as 0 debtorParticipant_alfin_banco_s.a.
added as 0 debtorParticipant_financiera_efectiva
added as 0 debtorParticipant_caja_trujillo
added as 0 debtorParticipant_caja_sullana
added as 0 debtorParticipant_caja_huancayo
added as 0 debtorParticipant_caja_ica
added as 0 creditorParticipant_banco_ripley
added as 0 creditorParticipant_alfin_banco_s.a.
added as 0 creditorParticipant_financiera_efectiva
added as 0 creditorParticipant_caja_sullana
added as 0 in_black_debtor
added as 0 in_black_creditor
dropping... same_customer_flag
dropping... day_interval


In [177]:
# model function
def fit_isolation_forest(input_data, args=None):
    rs = args["random_state"]
    c = args["contamination"]
   
    model = IsolationForest(random_state=rs, contamination=c, n_jobs=-1)
    model.fit(input_data)

    return model

In [178]:
AV_train.drop(id_cols, axis=1).columns.tolist()

['creditorParticipant_ratio',
 'creditorParticipant_currency_ratio',
 'creditorParticipant_channel_ratio',
 'creditorParticipant_responseCode_ratio',
 'creditorParticipant_debtorParticipant_ratio',
 'creditorParticipant_Weekday_ratio',
 'creditorParticipant_time_interval_ratio',
 'creditorParticipant_creditorCCI_ratio',
 'currency_ratio',
 'currency_channel_ratio',
 'currency_responseCode_ratio',
 'currency_debtorParticipant_ratio',
 'currency_Weekday_ratio',
 'currency_time_interval_ratio',
 'currency_creditorCCI_ratio',
 'channel_ratio',
 'channel_responseCode_ratio',
 'channel_debtorParticipant_ratio',
 'channel_Weekday_ratio',
 'channel_time_interval_ratio',
 'channel_creditorCCI_ratio',
 'responseCode_ratio',
 'responseCode_debtorParticipant_ratio',
 'responseCode_Weekday_ratio',
 'responseCode_time_interval_ratio',
 'responseCode_creditorCCI_ratio',
 'debtorParticipant_ratio',
 'debtorParticipant_Weekday_ratio',
 'debtorParticipant_time_interval_ratio',
 'debtorParticipant_credit

In [ ]:
#33 variables nuevas al 15/11
# 

In [ ]:
#drop id_cols(debtorId, creditorCCE, same_customer_flag)
if_model_90_new_vars = fit_isolation_forest(AV_train.drop(id_cols, axis=1), args_5)
if_model_90_old_vars = fit_isolation_forest(AV_train.drop(id_cols + new_columns + new_ratios, axis=1), args_5)
output_directory_path = ""
file_path_new_vars = os.path.join(output_directory_path, "mvp_1_model_90_opt_new_vars_081124.pickle")
file_path_old_vars = os.path.join(output_directory_path, "mvp_1_model_90_opt_old_vars_081124.pickle")
with open(file_path_new_vars, "wb") as handle:
    pickle.dump(if_model_90_new_vars, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(file_path_old_vars, "wb") as handle:
    pickle.dump(if_model_90_old_vars, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_model(file_path, file_name):
    full_file_path = os.path.join(file_path, file_name)
    with open(full_file_path, 'rb') as handle:
        loaded_model = pickle.load(handle)
    return loaded_model

In [ ]:
def predict_isolation_forest(input_data, model):
    print(input_data.columns.tolist())
    y_pred = model.predict(input_data)
    y_pred = [1 if pred == -1 else 0 for pred in y_pred]
    y_pred_series = pd.Series(data=y_pred, index=input_data.index).astype("uint8")
    return y_pred_series
def score_isolation_forest(input_data, model, days_back):
    print(input_data.columns.tolist())
    y_pred = model.decision_function(input_data)
    print("_______________________________")
    print("DAYS", days_back)
    print("MIN", y_pred.min())
    print("MAX", y_pred.max())
    if days_back == 7:
        min_value = -0.10404229802050258 # -0.11774102106942208
        max_value = 0.1814456418857644 #0.1602303765911105
    elif days_back == 90:
        min_value = -0.1268560699089064 # -0.11774102106942208
        max_value = 0.1737664642244749 #0.1602303765911105
    # norm_y_pred = (y_pred - min_value) / (max_value - min_value) * 99
    norm_y_pred = (max_value - y_pred) / (max_value - min_value) * 99
    # norm_y_pred = max(0, min(norm_y_pred, 99))
    # y_pred = [1 if pred == -1 else 0 for pred in y_pred]
    norm_y_pred = norm_y_pred.clip(min=0.0, max=99.0)
    # y_pred_series = pd.Series(data=y_pred, index=input_data.index).astype("float64")
    y_pred_series = pd.Series(data=norm_y_pred, index=input_data.index).astype("float64")
    y_pred_series = y_pred_series.round(8)
    return y_pred_series

In [ ]:
model_90 = load_model("", "mvp_1_model_90_opt_old_vars_230924.pickle")


In [ ]:
print("Number of input features:", model_90.n_features_in_)


In [ ]:
AV_90 = final
AV_90.drop(columns=id_cols, inplace=True)

variables = ['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio', 'Weekday_time_interval_ratio', 'Weekday_creditorCCI_ratio', 'time_interval_ratio', 'time_interval_creditorCCI_ratio', 'creditorCCI_ratio', 'hourSin', 'hourCos', 'dayOfYearSin', 'dayOfYearCos', 'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin', 'dayOfWeekCos','debtorParticipant_bcp','debtorParticipant_interbank','debtorParticipant_citibank','debtorParticipant_scotiabank','debtorParticipant_bbva','debtorParticipant_banco_de_la_nacion','debtorParticipant_comercio','debtorParticipant_banco_pichincha','debtorParticipant_banbif','debtorParticipant_crediscotia_financiera','debtorParticipant_mi_banco','debtorParticipant_gnb','debtorParticipant_banco_falabella','debtorParticipant_banco_ripley','debtorParticipant_alfin_banco_s.a.','debtorParticipant_financiera_oh','debtorParticipant_financiera_efectiva','debtorParticipant_caja_piura','debtorParticipant_caja_trujillo','debtorParticipant_caja_arequipa','debtorParticipant_caja_sullana','debtorParticipant_caja_cusco','debtorParticipant_caja_huancayo','debtorParticipant_caja_ica','debtorParticipant_invalid','creditorParticipant_bcp','creditorParticipant_interbank','creditorParticipant_citibank','creditorParticipant_scotiabank','creditorParticipant_bbva','creditorParticipant_banco_de_la_nacion','creditorParticipant_comercio','creditorParticipant_banco_pichincha','creditorParticipant_banbif','creditorParticipant_crediscotia_financiera','creditorParticipant_mi_banco','creditorParticipant_gnb','creditorParticipant_banco_falabella','creditorParticipant_banco_ripley','creditorParticipant_alfin_banco_s.a.','creditorParticipant_financiera_oh','creditorParticipant_financiera_efectiva','creditorParticipant_caja_piura','creditorParticipant_caja_trujillo','creditorParticipant_caja_arequipa','creditorParticipant_caja_sullana','creditorParticipant_caja_cusco','creditorParticipant_caja_huancayo','creditorParticipant_caja_ica','creditorParticipant_invalid', 'currency_soles', 'channel_banca_movil', 'channel_invalid', 'channel_web', 'responseCode_rejected']
if NEW_VARIABLES_FLAG:
    new_columns = ['f1d',
        'f7d',
        'f30d',
        'f90d',
        'f1d_to_creditor',
        'f7d_to_creditor',
        'f30d_to_creditor',
        'f90d_to_creditor',
        'unique_debtors_past_1d',
        'unique_debtors_past_7d',
        'unique_debtors_past_30d',
        'unique_debtors_past_90d',
        'prop_invalid_1d',
        'prop_banca_movil_1d',
        'prop_web_1d',
        'prop_atm_1d',
        'prop_invalid_7d',
        'prop_banca_movil_7d',
        'prop_web_7d',
        'prop_atm_7d',
        'prop_invalid_30d',
        'prop_banca_movil_30d',
        'prop_web_30d',
        'prop_atm_30d',
        'prop_invalid_90d',
        'prop_banca_movil_90d',
        'prop_web_90d',
        'prop_atm_90d',
    ]
    new_ratios = [
        "creditorParticipant_creditorId_ratio",
        "currency_creditorId_ratio",
        "channel_creditorId_ratio",
        "responseCode_creditorId_ratio",
        "debtorParticipant_creditorId_ratio",
        "Weekday_creditorId_ratio",
        "time_interval_creditorId_ratio",
        "creditorCCI_creditorId_ratio",
        "creditorId_ratio",
    ]
    variables += new_columns + new_ratios
# # Crear columnas si faltaran
for col in variables:
    if col not in AV_90.columns:
        AV_90[col] = 0

AV_90 = AV_90[variables].copy()

In [ ]:
y_pred_90 = predict_isolation_forest(AV_90, model_90,)
y_pred_90_same = pd.Series(data=[0] * len(final_same), index=final_same.index).astype("uint8")
y_pred_90 = pd.concat([y_pred_90, y_pred_90_same], axis=0)

y_score_90 = score_isolation_forest(AV_90, model_90, 90)
y_score_90_same = pd.Series(data=[0] * len(final_same), index=final_same.index).astype("float64")
y_score_90 = pd.concat([y_score_90, y_score_90_same], axis=0)


In [ ]:
xd = ['creditorParticipant_ratio', 'creditorParticipant_currency_ratio', 'creditorParticipant_channel_ratio', 'creditorParticipant_responseCode_ratio', 'creditorParticipant_debtorParticipant_ratio', 'creditorParticipant_Weekday_ratio', 'creditorParticipant_time_interval_ratio', 'creditorParticipant_creditorCCI_ratio', 'currency_ratio', 'currency_channel_ratio', 'currency_responseCode_ratio', 'currency_debtorParticipant_ratio', 'currency_Weekday_ratio', 'currency_time_interval_ratio', 'currency_creditorCCI_ratio', 'channel_ratio', 'channel_responseCode_ratio', 'channel_debtorParticipant_ratio', 'channel_Weekday_ratio', 'channel_time_interval_ratio', 'channel_creditorCCI_ratio', 'responseCode_ratio', 'responseCode_debtorParticipant_ratio', 'responseCode_Weekday_ratio', 'responseCode_time_interval_ratio', 'responseCode_creditorCCI_ratio', 'debtorParticipant_ratio', 'debtorParticipant_Weekday_ratio', 'debtorParticipant_time_interval_ratio', 'debtorParticipant_creditorCCI_ratio', 'Weekday_ratio', 'Weekday_time_interval_ratio', 'Weekday_creditorCCI_ratio', 'time_interval_ratio', 'time_interval_creditorCCI_ratio', 'creditorCCI_ratio', 'hourSin', 'hourCos', 'dayOfYearSin', 'dayOfYearCos', 'dayOfMonthSin', 'dayOfMonthCos', 'dayOfWeekSin', 'dayOfWeekCos', 'debtorParticipant_bcp', 'debtorParticipant_interbank', 'debtorParticipant_citibank', 'debtorParticipant_scotiabank', 'debtorParticipant_bbva', 'debtorParticipant_banco_de_la_nacion', 'debtorParticipant_comercio', 'debtorParticipant_banco_pichincha', 'debtorParticipant_banbif', 'debtorParticipant_crediscotia_financiera', 'debtorParticipant_mi_banco', 'debtorParticipant_gnb', 'debtorParticipant_banco_falabella', 'debtorParticipant_banco_ripley', 'debtorParticipant_alfin_banco_s.a.', 'debtorParticipant_financiera_oh', 'debtorParticipant_financiera_efectiva', 'debtorParticipant_caja_piura', 'debtorParticipant_caja_trujillo', 'debtorParticipant_caja_arequipa', 'debtorParticipant_caja_sullana', 'debtorParticipant_caja_cusco', 'debtorParticipant_caja_huancayo', 'debtorParticipant_caja_ica', 'debtorParticipant_invalid', 'creditorParticipant_bcp', 'creditorParticipant_interbank', 'creditorParticipant_citibank', 'creditorParticipant_scotiabank', 'creditorParticipant_bbva', 'creditorParticipant_banco_de_la_nacion', 'creditorParticipant_comercio', 'creditorParticipant_banco_pichincha', 'creditorParticipant_banbif', 'creditorParticipant_crediscotia_financiera', 'creditorParticipant_mi_banco', 'creditorParticipant_gnb', 'creditorParticipant_banco_falabella', 'creditorParticipant_banco_ripley', 'creditorParticipant_alfin_banco_s.a.', 'creditorParticipant_financiera_oh', 'creditorParticipant_financiera_efectiva', 'creditorParticipant_caja_piura', 'creditorParticipant_caja_trujillo', 'creditorParticipant_caja_arequipa', 'creditorParticipant_caja_sullana', 'creditorParticipant_caja_cusco', 'creditorParticipant_caja_huancayo', 'creditorParticipant_caja_ica', 'creditorParticipant_invalid', 'currency_soles', 'channel_banca_movil', 'channel_invalid', 'channel_web', 'responseCode_rejected', 'f1d', 'f7d', 'f30d', 'f90d', 'f1d_to_creditor', 'f7d_to_creditor', 'f30d_to_creditor', 'f90d_to_creditor', 'unique_debtors_past_1d', 'unique_debtors_past_7d', 'unique_debtors_past_30d', 'unique_debtors_past_90d', 'prop_invalid_1d', 'prop_banca_movil_1d', 'prop_web_1d', 'prop_atm_1d', 'prop_invalid_7d', 'prop_banca_movil_7d', 'prop_web_7d', 'prop_atm_7d', 'prop_invalid_30d', 'prop_banca_movil_30d', 'prop_web_30d', 'prop_atm_30d', 'prop_invalid_90d', 'prop_banca_movil_90d', 'prop_web_90d', 'prop_atm_90d', 'creditorParticipant_creditorId_ratio', 'currency_creditorId_ratio', 'channel_creditorId_ratio', 'responseCode_creditorId_ratio', 'debtorParticipant_creditorId_ratio', 'Weekday_creditorId_ratio', 'time_interval_creditorId_ratio', 'creditorCCI_creditorId_ratio', 'creditorId_ratio']


In [ ]:

new_columns = [
    'f1d',
    'f7d',
    'f30d',
    'f90d',
    'f1d_to_creditor',
    'f7d_to_creditor',
    'f30d_to_creditor',
    'f90d_to_creditor',
    'unique_debtors_past_1d',
    'unique_debtors_past_7d',
    'unique_debtors_past_30d',
    'unique_debtors_past_90d',
    'prop_invalid_1d',
    'prop_banca_movil_1d',
    'prop_web_1d',
    'prop_atm_1d',
    'prop_invalid_7d',
    'prop_banca_movil_7d',
    'prop_web_7d',
    'prop_atm_7d',
    'prop_invalid_30d',
    'prop_banca_movil_30d',
    'prop_web_30d',
    'prop_atm_30d',
    'prop_invalid_90d',
    'prop_banca_movil_90d',
    'prop_web_90d',
    'prop_atm_90d',
]
new_ratios = [
    "creditorParticipant_creditorId_ratio",
    "currency_creditorId_ratio",
    "channel_creditorId_ratio",
    "responseCode_creditorId_ratio",
    "debtorParticipant_creditorId_ratio",
    "Weekday_creditorId_ratio",
    "time_interval_creditorId_ratio",
    "creditorCCI_creditorId_ratio",
    "creditorId_ratio",
]

In [ ]:
a = [x for x in xd if x not in list((set(new_columns).union(set(new_ratios))))]

In [ ]:
a

In [ ]:
id_descriptive_features_list = \
    ['pk', 'debtorId', 'creditorCCI', 'creditorId', 'creditorIdCode', 'message_id', 'trace', 'instruction_id', 'run_id',
    'creationDate', 'creationTime', 'channel', 'currency', 'creditorParticipant', 'debtorParticipant',
    'debtorTypeOfPerson', 'transactionType', 'debtorIdCode', 'reasonCode', 'responseCode', 'same_customer_flag']


In [ ]:
original_values_set = AV_consolidado_original_values_set

In [ ]:
rate_features_list_90_new = AV_90.filter(like="_ratio").columns.to_list()

In [ ]:
rate_features_list_90_new

In [ ]:
AV_90.shape

In [ ]:
pd.concat([AV_90, final_same], axis=0).index

In [ ]:
original_values_set[id_descriptive_features_list]\
            .merge(pd.concat([AV_90, final_same], axis=0)[rate_features_list_90_new], how="inner", left_index=True, right_index=True)